In [1]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Normalization, StringLookup, IntegerLookup

In [2]:
# Setting the seed here is sufficient. 
# If you don't plan to use these starter code, make sure you add this cell.

SEED = 42

import os
os.environ['TF_CUDNN_DETERMINISTIC'] = '1'

import random 
random.seed(SEED)

import numpy as np
np.random.seed(SEED)

import tensorflow as tf
tf.random.set_seed(SEED)

In [3]:
import pandas as pd
df = pd.read_csv('hdb_price_prediction.csv')
df

,month,year,full_address,nearest_stn,dist_to_nearest_stn,dist_to_dhoby,degree_centrality,eigenvector_centrality,flat_model_type,remaining_lease_years,floor_area_sqm,storey_range,resale_price
0,1,2017,406 ANG MO KIO AVENUE 10,Ang Mo Kio,1.007264,7.006044,0.016807,0.006243,"2 ROOM, Improved",61.333333,44.0,10 TO 12,232000.0
1,1,2017,108 ANG MO KIO AVENUE 4,Ang Mo Kio,1.271389,7.983837,0.016807,0.006243,"3 ROOM, New Generation",60.583333,67.0,01 TO 03,250000.0
2,1,2017,602 ANG MO KIO AVENUE 5,Yio Chu Kang,1.069743,9.090700,0.016807,0.002459,"3 ROOM, New Generation",62.416667,67.0,01 TO 03,262000.0
3,1,2017,465 ANG MO KIO AVENUE 10,Ang Mo Kio,0.946890,7.519889,0.016807,0.006243,"3 ROOM, New Generation",62.083333,68.0,04 TO 06,265000.0
4,1,2017,601 ANG MO KIO AVENUE 5,Yio Chu Kang,1.092551,9.130489,0.016807,0.002459,"3 ROOM, New Generation",62.416667,67.0,01 TO 03,265000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
133407,6,2022,877 YISHUN STREET 81,Khatib,0.475885,12.738721,0.016807,0.000968,"EXECUTIVE, Maisonette",64.583333,145.0,07 TO 09,810000.0
133408,1,2022,633 YISHUN STREET 61,Khatib,0.774113,13.229106,0.016807,0.000968,"MULTI-GENERATION, Multi Generation",65.000000,164.0,04 TO 06,785000.0
133409,2,2022,633 YISHUN STREET 61,Khatib,0.774113,13.229106,0.016807,0.000968,"MULTI-GENERATION, Multi Generation",64.916667,171.0,04 TO 06,842000.0
133410,2,2022,632 YISHUN STREET 61,Khatib,0.700595,13.222912,0.016807,0.000968,"MULTI-GENERATION, Multi Generation",64.750000,164.0,10 TO 12,845000.0


In [4]:
# The functions in this cell are adapted from https://keras.io/examples/structured_data/structured_data_classification_from_scratch/
# It is the same link as the one mentioned in the question paper (Q1b)

def dataframe_to_dataset(dataframe):
    dataframe = dataframe.copy()
    labels = dataframe.pop("resale_price")
    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
    ds = ds.shuffle(buffer_size=len(dataframe))
    return ds


def encode_numerical_feature(feature, name, dataset):
    # Create a Normalization layer for our feature
    normalizer = Normalization()

    # Prepare a Dataset that only yields our feature
    feature_ds = dataset.map(lambda x, y: x[name])
    feature_ds = feature_ds.map(lambda x: tf.expand_dims(x, -1))

    # Learn the statistics of the data
    normalizer.adapt(feature_ds)

    # Normalize the input feature
    encoded_feature = normalizer(feature)
    return encoded_feature


def encode_categorical_feature(feature, name, dataset, is_string):
    lookup_class = StringLookup if is_string else IntegerLookup
    # Create a lookup layer which will turn strings into integer indices
    lookup = lookup_class(output_mode="binary") # NOTE: as mentioned in the question paper, this actually does one-hot encoding. You could replace 'binary' with 'one_hot' if you wish to.

    # Prepare a Dataset that only yields our feature
    feature_ds = dataset.map(lambda x, y: x[name])
    feature_ds = feature_ds.map(lambda x: tf.expand_dims(x, -1))

    # Learn the set of possible string values and assign them a fixed integer index
    lookup.adapt(feature_ds)

    # Turn the string input into integer indices
    encoded_feature = lookup(feature)
    return encoded_feature


In [5]:
from keras import backend as K

def r2(y_true, y_pred): 
    '''
    # Obtained from https://jmlb.github.io/ml/2017/03/20/CoeffDetermination_CustomMetric4Keras/
    # TODO: you have to find out how to use it in your code
    '''
    SS_res = K.sum(K.square( y_true - y_pred )) 
    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) ) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

In [6]:
# Split data

train_dataframe = df[df['year'] <= 2020]
val_dataframe = df[df['year'] > 2020]

train_ds = dataframe_to_dataset(train_dataframe)
val_ds = dataframe_to_dataset(val_dataframe)


train_ds = train_ds.batch(256)
val_ds = val_ds.batch(256)


In [7]:
# Categorical features encoded as integers
month = keras.Input(shape=(1,), name="month", dtype="int64")
month_encoded = encode_categorical_feature(month, "month", train_ds, False)

# Categorical feature encoded as string
flat_model_type = keras.Input(shape=(1,), name="flat_model_type", dtype="string")
storey_range = keras.Input(shape=(1,), name="storey_range", dtype="string")

flat_model_type_encoded = encode_categorical_feature(flat_model_type, "flat_model_type", train_ds, True)
storey_range_encoded = encode_categorical_feature(storey_range, "storey_range", train_ds, True)


# Numerical features
dist_to_nearest_stn = keras.Input(shape=(1,), name="dist_to_nearest_stn")
floor_area_sqm = keras.Input(shape=(1,), name="floor_area_sqm")
dist_to_dhoby = keras.Input(shape=(1,), name="dist_to_dhoby")
degree_centrality = keras.Input(shape=(1,), name="degree_centrality")
eigenvector_centrality = keras.Input(shape=(1,), name="eigenvector_centrality")
remaining_lease_years = keras.Input(shape=(1,), name="remaining_lease_years")

dist_to_nearest_stn_encoded = encode_numerical_feature(dist_to_nearest_stn, "dist_to_nearest_stn", train_ds)
floor_area_sqm_encoded = encode_numerical_feature(floor_area_sqm, "floor_area_sqm", train_ds)
dist_to_dhoby_encoded = encode_numerical_feature(dist_to_dhoby, "dist_to_dhoby", train_ds)
degree_centrality_encoded = encode_numerical_feature(degree_centrality, "degree_centrality", train_ds)
eigenvector_centrality_encoded = encode_numerical_feature(eigenvector_centrality, "eigenvector_centrality", train_ds)
remaining_lease_years_encoded = encode_numerical_feature(remaining_lease_years, "remaining_lease_years", train_ds)


all_inputs = [
    month,
    storey_range,
    flat_model_type,
    floor_area_sqm,
    remaining_lease_years,
    degree_centrality,
    eigenvector_centrality,
    dist_to_nearest_stn,
    dist_to_dhoby,
]


all_features = layers.concatenate(
    [
        month_encoded,
        storey_range_encoded,
        flat_model_type_encoded,
        floor_area_sqm_encoded,
        remaining_lease_years_encoded,
        degree_centrality_encoded,
        eigenvector_centrality_encoded,
        dist_to_nearest_stn_encoded,
        dist_to_dhoby_encoded,
    ]
)

In [8]:
dense_layer = layers.Dense(1, activation="linear")(all_features)
model = keras.Model(all_inputs, dense_layer)
opt=keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=opt, loss='mse', metrics=[tf.keras.metrics.MeanSquaredError(), r2])
history = model.fit(train_ds, epochs=50, validation_data=val_ds)

Epoch 1/50


c:\Users\Atul\AppData\Local\Programs\Python\Python38\lib\site-packages\keras\engine\functional.py:566: UserWarning: Input dict contained keys ['year', 'full_address', 'nearest_stn'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


342/342 [==============================] - 3s 7ms/step - loss: 219585937408.0000 - mean_squared_error: 219585937408.0000 - r2: -8.3474 - val_loss: 301486735360.0000 - val_mean_squared_error: 301486735360.0000 - val_r2: -10.0832
Epoch 2/50
342/342 [==============================] - 2s 6ms/step - loss: 219584561152.0000 - mean_squared_error: 219584561152.0000 - r2: -8.3520 - val_loss: 301485064192.0000 - val_mean_squared_error: 301485064192.0000 - val_r2: -10.0722
Epoch 3/50
342/342 [==============================] - 3s 6ms/step - loss: 219583217664.0000 - mean_squared_error: 219583217664.0000 - r2: -8.3501 - val_loss: 301483458560.0000 - val_mean_squared_error: 301483458560.0000 - val_r2: -10.1026
Epoch 4/50
342/342 [==============================] - 3s 6ms/step - loss: 219581857792.0000 - mean_squared_error: 219581857792.0000 - r2: -8.3437 - val_loss: 301481951232.0000 - val_mean_squared_error: 301481951232.0000 - val_r2: -10.0800
Epoch 5/50
342/342 [==============================] - 3

In [9]:
dense_layer = layers.Dense(1, activation="linear")(all_features)
model = keras.Model(all_inputs, dense_layer)
opt=keras.optimizers.SGD(learning_rate=0.01)
model.compile(optimizer=opt, loss='mse', metrics=[tf.keras.metrics.MeanSquaredError(), r2])
history = model.fit(train_ds, epochs=50, validation_data=val_ds)

Epoch 1/50
342/342 [==============================] - 21s 38ms/step - loss: 18728673280.0000 - mean_squared_error: 18728673280.0000 - r2: 0.1966 - val_loss: 13789992960.0000 - val_mean_squared_error: 13789992960.0000 - val_r2: 0.4958
Epoch 2/50
342/342 [==============================] - 17s 40ms/step - loss: 5481924096.0000 - mean_squared_error: 5481924096.0000 - r2: 0.7678 - val_loss: 12796292096.0000 - val_mean_squared_error: 12796292096.0000 - val_r2: 0.5320
Epoch 3/50
342/342 [==============================] - 18s 46ms/step - loss: 5026647552.0000 - mean_squared_error: 5026647552.0000 - r2: 0.7875 - val_loss: 12325086208.0000 - val_mean_squared_error: 12325086208.0000 - val_r2: 0.5501
Epoch 4/50
342/342 [==============================] - 28s 63ms/step - loss: 4776961536.0000 - mean_squared_error: 4776961536.0000 - r2: 0.7976 - val_loss: 12102072320.0000 - val_mean_squared_error: 12102072320.0000 - val_r2: 0.5574
Epoch 5/50
342/342 [==============================] - 21s 46ms/step - 

KeyboardInterrupt: 

In [10]:
hidden_layer = layers.Dense(10, activation="relu")(all_features)
dense_layer = layers.Dense(1, activation="linear")(hidden_layer)
model = keras.Model(all_inputs, dense_layer)
opt=keras.optimizers.Adam(learning_rate=0.08)
model.compile(optimizer=opt, loss='mse', metrics=[tf.keras.metrics.MeanSquaredError(), r2])
history = model.fit(train_ds, epochs=50, validation_data=val_ds)

Epoch 1/50
342/342 [==============================] - 4s 8ms/step - loss: 203814240256.0000 - mean_squared_error: 203814240256.0000 - r2: -7.6865 - val_loss: 241260724224.0000 - val_mean_squared_error: 241260724224.0000 - val_r2: -7.8587
Epoch 2/50
342/342 [==============================] - 3s 6ms/step - loss: 117010554880.0000 - mean_squared_error: 117010554880.0000 - r2: -3.9765 - val_loss: 113441415168.0000 - val_mean_squared_error: 113441415168.0000 - val_r2: -3.1711
Epoch 3/50
342/342 [==============================] - 3s 6ms/step - loss: 35444862976.0000 - mean_squared_error: 35444862976.0000 - r2: -0.5061 - val_loss: 36721967104.0000 - val_mean_squared_error: 36721967104.0000 - val_r2: -0.3447
Epoch 4/50
342/342 [==============================] - 2s 6ms/step - loss: 8698674176.0000 - mean_squared_error: 8698674176.0000 - r2: 0.6322 - val_loss: 16459524096.0000 - val_mean_squared_error: 16459524096.0000 - val_r2: 0.3977
Epoch 5/50
342/342 [==============================] - 3s 6ms